## 25. 새로운 API

### 이 장에서 다루는 내용
- 부드러운 애니메이션 만들기
- 파일로 작업하기
- 웹 워커를 이용한 백그라운드 자바스크립트


### requestAnimationFrame()
``` javascript 
    !(function() {
      let start = new Date().getTime();
      let callback = function() {
      let ts = new Date().getTime();
        if (ts - 1000 > start) {
          // console.log('End');
        }
        else {
          console.log(ts);
          requestAnimationFrame(callback);
        }
      }
      requestAnimationFrame(callback);
})();
```
- 성능이 당연히 setInterval 보다 좋기 때문에, setInterval 을 쓰지 않고 requestAnimationFrame 을 쓰는 것이 좋다.
- 백그라운드 동작 및 비활성화시 중지(성능 최적화)
- 최대 1ms(1/1000s)로 제한되며 1초에 60번 동작
- requestAnimationFrame 함수의 이름이 브라우저마다 다른 경우가 있고, 버젼에 따라서 지원이 되지 않을 수도 있다. 그러므로 코드 상단에 아래와 같은 코드를 삽입하는 것이 좋다. (아래 코드)

``` javascript
window.requestAnimFrame = (function(callback) {
      return window.requestAnimationFrame ||
      window.webkitRequestAnimationFrame ||
      window.mozRequestAnimationFrame ||
      window.oRequestAnimationFrame ||
      window.msRequestAnimationFrame ||
      function(callback) { window.setTimeout(callback, 1000 / 60); };
})();
```
    
### 페이지 가시성

- Document.visibilityState 읽기 전용 property로, 이 element가 현재 표시된 컨텍스트를 나타내는 document의 가시성을 반환합니다. document가 background 또는 보이지 않는 탭(다른 탭)에 있는지, 또는 pre-rendering을 위해 로드 된 것인지를 아는 것은 유용합니다. 가능한 값은 다음과 같습니다.
    - 'Document.visible' : 페이지 내용은 적어도 부분적으로 보일 수 있습니다. 실제로 이는 페이지가 최소화 되지 않은 창(브라우저)에서의 선택된 탭 을 의미 합니다.
    - 'Document.hidden' : 페이지 내용은 사용자에게 표시되지 않습니다. 실제로 이는 document background-tap(다른 탭)이거나, 최소화 된 창의 일부이거나, OS 화면 잠금이 활성 상태임을 의미합니다.
    읽기 전용 속성은 페이지가 숨겨졌는지 여부를 Boolean 값으로 나타냅니다.
    - 'Document.prerender' : 페이지 내용이 pre-rendering되어 사용자에게 보이지 않습니다 (document.hidden 목적으로 숨겨진 것으로 간주 합니다.). document는이 상태에서 시작될 수 있지만, 절대로 다른 값에서 이 값으로 전환되지는 않습니다. 참고 : 브라우저 지원은 선택 사항입니다.
    - 'Document.unloaded' : 페이지가 메모리에서 로드되지 않았습니다. 참고 : 브라우저 지원은 선택 사항입니다. 이 property의 값이 변경되면 visibilitychange 이벤트가 Document로 전송됩니다.


```javascript
document.addEventListener("visibilitychange", function() {
  console.log(document.hidden);
  // 숨김 여부가 변했을 때의 행동
});
```

### Geolocation API
- navigator.geolocation을 통해 접근합니다. 이 때, 사용자의 브라우저는 위치 정보 접근 권한을 요청하게 되고, 사용자가 허가할 경우 현재 장치에서 사용 가능한 최선의 방법(GPS, WiFi, ...)을 통해 위치를 알아냅니다.

- 위의 과정이 끝난 후, 코드에서는 몇 가지 다른 방법으로 위치 정보를 가져올 수 있습니다.
- GPS에 대한 정보는 window.navigator 객체 안에 들어있습니다.
- Geolocation.getCurrentPosition(): 장치의 현재 위치를 가져옵니다.
- Geolocation.watchPosition(): 장치의 위치가 바뀔 때마다, 자동으로 새로운 위치를 사용해 호출할 처리기 함수를 등록합니다.

In [ ]:
function getLocation() {
  if (navigator.geolocation) { // GPS를 지원하면
    navigator.geolocation.getCurrentPosition(function(position) {
      alert(position.coords.latitude + ' ' + position.coords.longitude);
    }, function(error) {
      console.error(error);
    }, {
      enableHighAccuracy: false,
      maximumAge: 0,
      timeout: Infinity
    });
  } else {
    alert('GPS를 지원하지 않습니다');
  }
}
getLocation();

## File Api
File 인터페이스는 파일에 대한 정보를 제공하고, 웹 페이지의 JavaScript가 해당 내용에 접근할 수 있는 방법을 제공합니다.


File 객체는 보통 FileList 객체에서 가져올 수 있습니다. 그리고 FileList 객체는 사용자가 ```<input>``` 요소에서 파일을 선택했을 때, 드래그 앤 드롭 작업의 DataTransfer 객체에서, 그리고 HTMLCanvasElement의 mozGetAsFile() API에서 반환합니다.

- FileList : 파일 리스트
- File : 파일 데이터
- FileReader : 파일 읽기
- Blob : 바이트 데이터

- 예제 ) https://web.dev/read-files/

In [ ]:
<input type="file" id="file-selector" multiple>
<script>
  const fileSelector = document.getElementById('file-selector');
  fileSelector.addEventListener('change', (event) => {
    const fileList = event.target.files;
    console.log(fileList);
  });
</script>

### input 의 속성
- input[type=file] 은 value, accept, capture, files, multiple 속성을 갖을 수 있다.
    - value [DOMString] : 파일 경로
    - accept [MIME] : 사용 가능한 파일 종류
    - capture [string] : 파일 캡처 방법
    - multiple [boolean] : 여러 파일 선택 여부
    - files [FileList] : 선택된 파일들
    
    
### FileReader
- FileReader 은 File 이나 Blob 의 내용을 읽을 수 있게 도와준다.
- 보안상 직접적인 Local Storage 에는 접근할 수 없다.
- FileReader 에는 4가지 방법으로 파일을 전달 할 수 있다.
    - readAsArrayBuffer(file|blob) [ArrayBuffer]
    - readAsBinaryString(file|blob) [0..255 범위의 문자열]
    - readAsDataURL(file|blob) [Base64]
    - readAsText(file|blob) [UTF-16|UTF-8 문자열]

https://medium.com/@pks2974/file-api-%EC%A0%95%EB%A6%AC%ED%95%98%EA%B8%B0-729fa6a3a0ba

## Drag&Drop : File API
- HTML 드래그 앤 드롭은 DOM event model 과 drag events 를  mouse events 로부터 상속받습니다. 보통 드래그는 사용자가 draggable 요소를 마우스로 선택하고, 마우스 포인터를 droppable 요소로 가져가 마우스 버튼을 때는 것으로 이루어집니다. 드래그하는 도중에 많은 이벤트가 발생하고, 몇몇 이벤트는 여러번 발생하기도 합니다. ( drag와 dragover).

예제) https://jsbin.com/hiqasek/edit?html,js,output

## 웹타이밍
- Navigation Timing API는 웹 사이트의 성능을 측정하는 데 사용할 수 있는 데이터를 제공합니다. 같은 목적에 사용했던 다른 JavaScript 기반 메커니즘과 다르게 이 API는 더 유용하고 정확한 종단 간(end-to-end) 대기 시간(latency)을 제공할 수 있습니다.

In [ ]:
function onLoad() { 
  var now = new Date().getTime();
  var page_load_time = now - performance.timing.navigationStart;
  console.log("User-perceived page loading time: " + page_load_time);
}

In [ ]:
// 어떤 페이지를 로딩하는 데 필요한 전체 시간 계산하기.
var perfData = window.performance.timing; 
var pageLoadTime = perfData.loadEventEnd - perfData.navigationStart;

// 요청 응답 시간 계산하기.
var connectTime = perfData.responseEnd - perfData.requestStart;

- PerformanceTiming 인터페이스로 접근할 수 있는 밀리 초 단위로 주어진 측정된 이벤트가 많이 있습니다. 발생하는 순서로 된 이벤트 목록입니다.

    - navigationStart
    - unloadEventStart
    - unloadEventEnd
    - redirectStart
    - redirectEnd
    - fetchStart
    - domainLookupStart
    - domainLookupEnd
    - connectStart
    - connectEnd
    - secureConnectionStart
    - requestStart
    - responseStart
    - responseEnd
    - domLoading
    - domInteractive
    - domContentLoadedEventStart
    - domContentLoadedEventEnd
    - domComplete
    - loadEventStart
    - loadEventEnd

## 웹워커
- 웹워커는 Background Thread에서 스크립트를 실행하는 방법이다. UI Thread와는 별개의 Thread를 실행시켜 사용자 인터페이스(UI)를 방해하지 않고 작업을 수행할 수 있다.

- Thread란 어떠한 프로그램 내에서, 특히 프로세스 내에서 실행되는 흐름의 단위를 말한다. 일반적으로 한 프로그램은 하나의 스레드를 가지고 있지만, 프로그램 환경에 따라 둘 이상의 스레드를 동시에 실행할 수 있다. 이러한 실행 방식을 멀티스레드(multithread)라고 한다. ( 출처 : 위키백과 )

### 웹 워커 활용
- 매우 복잡한 수학적 계산 작업
- 원격지에 있는 리소스에 대한 액세스 작업(또는 로컬 스토로지를 액세스 하는 경우)
- 백그라운드에서 오랜시간 작업해야 하는 경우
- UI 쓰레드에 방해 없이 지속적으로 수행해야 하는 작업 등

In [ ]:
if ( !!window.Worker ) {
    alert('웹 워커를 지원하는 브라우저입니다.');
} else {
    alert( '웹 워커를 지원하지 않는 브라우저입니다.' );
}

### Worker 실행 파일 ( worker.js ) 작성
워커가 실행할 스크립트는 별도의 자바스크립트 파일( worker.js )을 만들고 작성해야 한다. self는 워커 내부의 worker 전역 스코프에 대한 참조이다.

In [ ]:
// worker 메시지 수신 listener
self.onmessage = function( e ) {
    console.log( 'Worker가 받은 메시지 ', e.data );

    // 1초 후에 호출한 페이지에 데이터를 보낸다.
    setTimeout( function() {
        postMessage( 'Worker Value' );
    }, 1000 );
};

### Worker 호출
Worker객체를 만든 후 생성한 worker 파일의 이름을 매개변수로  넘겨주어야 한다.

In [ ]:
var worker = new Worker( 'worker.js' );
worker.postMessage( '워커 실행' );  // 워커에 메시지를 보낸다.

// 메시지는 JSON구조로 직렬화 할 수 있는 값이면 사용할 수 있다. Object등 
// worker.postMessage( { name : '302chanwoo' } );

// 워커로 부터 메시지를 수신한다.
worker.onmessage = function( e ) {
    console.log('호출 페이지 - ', e.data );
};

## 워커 종료하기

``` javascript 
worker.terminate();
```

``` html
<button id="btnStartWorker" class="btn">워커 시작</button>
<button id="btnStopWorker" class="btn">워커 중지</button>
<div id="output"></div> 
```

In [2]:
// worker.js
  var btnStartWorker = document.getElementById( 'btnStartWorker' );     // worker 실행 버튼
    var btnStopWorker = document.getElementById( 'btnStopWorker' );       // worker 중지 버튼
    var output = document.getElementById( 'output' );                     // 받은 메시지 출력
    var worker;                                                           // worker

    btnStartWorker.addEventListener( 'click', startWorker );
    btnStopWorker.addEventListener( 'click',stopWorker );

    // worker 실행
    function startWorker() {

      // Worker 지원 유무 확인
      if ( !!window.Worker ) {

        // 실행하고 있는 워커 있으면 중지시키기
        if ( worker ) {
          stopWorker();
        }

        worker = new Worker( 'worker.js' );
        worker.postMessage( '워커 실행' );    // 워커에 메시지를 보낸다.

        // 메시지는 JSON구조로 직렬화 할 수 있는 값이면 사용할 수 있다. Object등
        // worker.postMessage( { name : '302chanwoo' } );

        // 워커로 부터 메시지를 수신한다.
        worker.onmessage = function( e ) {
          console.log('호출 페이지 - ', e.data );
          output.innerHTML += e.data;
        };
      }

    }

    // worker 중지
    function stopWorker() {

      if ( worker ) {
        worker.terminate();
        worker = null;
      }

    }

ReferenceError: document is not defined

http://blog.302chanwoo.com/2016/08/webworker/